In [684]:
drive_path = '/Volumes/Brain2016'
# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import sys
from scipy.stats.stats import pearsonr
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import seaborn
# import time as tm
import warnings

with warnings.catch_warnings():
    warnings.simplefilter('error')

manifest_path = os.path.join(drive_path,'BrainObservatory','manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)

Using matplotlib backend: MacOSX


In [635]:
expMetaData = pd.DataFrame(boc.get_ophys_experiments())

## Load Data

In [1]:
expContainerID = expMetaData.id[5]
# expContainerID = 501940850
exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
rSpd, t_rSpd = exp.get_running_speed()
t_dF, dF = exp.get_dff_traces()
mCor = exp.get_motion_correction()

nanMask = ~np.isnan(rSpd)
rSpd = rSpd[nanMask]
rSpd = abs(rSpd)
dF = dF[:,nanMask]

NameError: name 'expMetaData' is not defined

#### Grab motion correction data (Jose)

In [619]:
expContainerID = expMetaData.id[5]
exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
t_dF, dF = exp.get_dff_traces()
mCor = exp.get_motion_correction()

fig,ax = plt.subplots(nrows = 3)

ax[0].plot(dF[0])
ax[1].plot(mCor.x_motion)
ax[2].plot(mCor.y_motion)

# convSig = signal.fftconvolve(dF[6],mCor.x_motion,'full')
# relevInds = np.arange(len(convSig)/2-10*30,len(convSig)/2+10*30)
plt.figure()
plt.scatter(dF[0],np.sqrt(mCor.x_motion**2 + mCor.y_motion**2))
# plt.plot(convSig[relevInds])

cells = boc.get_cell_specimens(experiment_container_ids = [expMetaData.experiment_container_id[5]])
# cells = pd.DataFrame.from_records(cells)

exp.list_stimuli()
stimTab = exp.get_stimulus_table('static_gratings')


from allensdk.brain_observatory.drifting_gratings import DriftingGratings
sg = StaticGratings(exp)

## Motion Detection Algorithm

In [933]:
rThresh = 15
runStart = np.where(rSpd > rThresh); runStart = runStart[0];

firstInd = 0
secondInd = 1
runBouts = []
while secondInd < len(runStart):
    runEndFound = False
    if (runStart[secondInd] - runStart[firstInd]) <= 30: # run lengths < 1s long
        secondInd += 1
    elif (runStart[secondInd] - runStart[firstInd]) <= 60: # an appropriately long run bout
        iterInd1 = secondInd
        iterInd2 = iterInd1 + 1
        while ~runEndFound:
            if iterInd2 == len(runStart):
                runEndFound = True
                secondInd = iterInd2 + 1
                break
            if (runStart[iterInd2] - runStart[iterInd1]) <= 30:
                iterInd2 += 1
            elif (runStart[iterInd2] - runStart[iterInd1]) <= 60:
                iterInd1 = iterInd2 - 1
                iterInd2 += 1
            else:
                runEndFound = True
                runBouts.append(runStart[firstInd])
                firstInd = iterInd2
                secondInd = iterInd2 + 1
                break
    else:
        firstInd = secondInd 
        secondInd += 1

### Plot motion detection algorithm

In [858]:
fig,ax = plt.subplots()
print(len(runBouts))
ax.plot(rSpd,alpha = 0.7)
ax.scatter(runBouts,np.ones(len(runBouts))*rThresh,s=50,c='red')
ax.set_xlim(0, 50000)
ax.set_ylim(0, 50)

65


(0, 50)

## Motion-triggered cell info

In [934]:
locDF = pd.DataFrame(index=np.arange(len(runBouts)),columns=np.arange(dF.shape[0])) # runBouts, cellNum + 1 locomotion
for cell in range(dF.shape[0]):
    for locOnset in runBouts:
        locDF.loc[np.where(runBouts == locOnset)[0][0],cell] = dF[cell][locOnset-90:locOnset+90]
        
locDF['dx'] = ''
for locOnset in runBouts:
    locDF.loc[np.where(runBouts == locOnset)[0][0],'dx'] = rSpd[locOnset-90:locOnset+90]

In [935]:
fig,ax = plt.subplots(nrows = 3)
ax[0].plot(locDF.loc[:,'dx'].mean(),label='Running Speed (cm/s)')
ax[1].plot(locDF.loc[:,37].mean(),label='$\Delta$F/F$_0$')
ax[2].errorbar(x = 0, y = locDF.loc[:,37].mean()[:90].mean(), yerr = locDF.loc[:,37].mean()[:90].std()/np.sqrt(locDF.shape[0]),label="Mean Pre-Run $\Delta$F/F$_0$")
ax[2].errorbar(x = 1, y = locDF.loc[:,37].mean()[90:].mean(), yerr = locDF.loc[:,37].mean()[90:].std()/np.sqrt(locDF.shape[0]),label="Mean Post-Run $\Delta$F/F$_0$")
for currAx in ax:
    currAx.legend(loc='best',fontsize=12)
ax[2].set_xlim(-0.5, 1.5)

(-0.5, 1.5)

In [931]:
locDF

,0,1,2,3,4,5,6,7,8,9,...,163,164,165,166,167,168,169,170,171,dx
1,"[-0.0703276, -0.0165346, -0.0710597, 0.0089616...","[-0.0609032, 0.00494042, -0.0700306, -0.035435...","[0.0389211, 0.0841815, -0.0853005, -0.0516435,...","[-0.0769755, -0.134087, -0.0787261, -0.0429409...","[-0.0299763, -0.076004, 0.0307096, -0.0638115,...","[0.00526243, -0.0299564, -0.113003, -0.0381493...","[-0.069997, -0.0395356, -0.00480972, 0.0380858...","[-0.0618847, 0.00900037, 0.0230158, 0.0338304,...","[0.00301644, -0.047115, -0.0112465, -0.0455651...","[-0.0968524, -0.0847863, -0.00597365, -0.04499...",...,"[-0.0458153, 0.00205502, 0.0143001, 0.030487, ...","[-0.0589301, 0.0159387, -0.00590442, -0.049729...","[0.0537863, 0.0517514, 0.0462365, 0.0547873, 0...","[0.0183676, 0.0359397, 0.0550625, -0.025502, 0...","[0.107724, -0.0199485, 0.0208837, -0.0544617, ...","[-0.126657, -0.0376404, -0.0502568, -0.0700408...","[-0.108432, -0.102242, -0.0914072, -0.00301819...","[0.0896952, -0.0535691, 0.0251476, -0.117239, ...","[-0.00427333, 0.0241338, 0.0402246, -0.0230908...","[2.25953745842, 1.24172115326, 1.5889903307, 0..."
2,"[0.0681994, 0.021182, 0.129948, 0.0108434, 0.0...","[0.131281, 0.0755865, -0.00553767, -0.0184144,...","[0.00414602, -0.0330964, -0.0248117, 0.0096512...","[0.0226372, 0.0410723, 0.0524209, 0.0502043, -...","[0.0415793, 0.0177493, 0.00481134, 0.0357968, ...","[0.0911647, 0.0164484, 0.130435, 0.056222, 2.1...","[-0.0535273, 0.0115366, -0.016445, 0.0108998, ...","[0.000756261, 0.0348416, 0.0141762, 0.00552549...","[0.0789275, -0.0460124, 0.00927661, 0.0347483,...","[0.109724, 0.0674846, -0.0244396, -0.10459, -0...",...,"[0.130134, 0.132335, 0.166798, 0.061223, 0.064...","[0.13207, 0.0536963, 0.0391937, -0.0177866, 0....","[0.140333, 0.146802, 0.0893518, 0.0383238, 0.0...","[0.0784461, 0.0216528, 0.0137339, 0.0278217, 0...","[-0.0448107, -0.117617, -0.128818, 0.013922, 0...","[0.0381047, 0.0326575, 0.0824286, 0.0243641, 0...","[0.0192578, -0.00586986, -0.0117374, 0.018971,...","[0.166184, 0.119507, 0.401169, 0.34997, 0.2365...","[0.00567251, 0.0323832, 0.0406637, -0.0787317,...","[6.48371887207, 0.247044220567, 0.501441478729..."
3,"[-0.0201212, 0.0134575, 0.067837, 0.0152652, 0...","[0.0973412, 0.0933303, 0.0138399, -0.119847, -...","[-0.0652042, 0.0470931, -0.020297, 0.0344054, ...","[0.0500683, 0.0442374, 0.0734489, 0.0527719, -...","[-0.0440736, -0.0177968, -0.0159273, -0.000879...","[-0.0318378, 0.0171834, -0.0779759, 0.0419705,...","[-0.0431807, 0.0262442, -0.0162484, 0.00665472...","[-0.0191, -0.0442863, -0.0048389, 0.0932085, -...","[0.0210123, -0.00919688, -0.0600353, 0.0421453...","[0.0299324, -0.0306902, 0.0558592, -0.00070162...",...,"[0.0286188, 0.0278364, 0.109501, 0.184824, 0.0...","[0.0978261, 0.113209, 0.00829017, 0.0312762, 0...","[0.101571, -0.0251124, 0.0435811, 0.0650877, 0...","[0.065462, 0.11393, -0.0155142, -0.0260572, 0....","[0.00085416, 0.0941391, 0.0350333, 0.0677836, ...","[0.0632519, 0.0754108, -0.0192909, 0.131715, 0...","[0.0125676, -0.0891826, 0.0423511, -0.0364564,...","[0.0529718, 0.136201, -0.0248913, 0.0387363, 0...","[0.0162426, 0.0193332, 0.0255022, -0.0050377, ...","[0.31860601902, 0.12812705338, 0.448230206966,..."
4,"[-0.0451395, -0.104373, 0.0190925, -0.0502661,...","[-0.0750009, 0.017355, -0.00734902, -0.130207,...","[0.0235038, 0.00789507, -0.0164373, -0.0057775...","[-0.14861, -0.0899839, -0.0214925, -0.0153935,...","[-0.140144, 0.0258756, -0.00772145, 0.0618584,...","[0.00523796, -0.0971859, -0.0178218, -0.072671...","[-0.0352865, 0.00422521, 0.0357224, 0.0295717,...","[-0.0159399, 0.00366699, 0.0297734, -0.0741326...","[-0.0572402, -0.117545, -0.0138331, 0.0421165,...","[-0.0701753, -0.0334485, -0.00221027, -0.06749...",...,"[-0.0972956, -0.0352041, -0.0846734, 0.0074579...","[0.0269149, -0.0140688, 0.0224641, -0.0293495,...","[0.0918911, -0.0199291, 0.0248419, 0.0399543, ...","[0.0678479, -0.00146977, -0.0221432, -0.064555...","[-0.00263494, -0.0300902, -0.126397, -0.022735...","[-0.0113811

In [176]:
fig,ax = plt.subplots()
for j in range(locDF.shape[1]-1):
    ax.plot(locDF.loc[:,j].mean(),label = j)
    ax.legend(loc = 'upper right', fontsize = 10)
    ax.set_ylim(-.2,0.6)
    plt.pause(0.3)
#     ax.legend_.remove()
    if j%6 == 5:
        ax.cla()
ax.close()


# plt.plot(locDF.stack().mean())

AttributeError: 'AxesSubplot' object has no attribute 'close'

In [157]:
fig,ax = plt.subplots()

cellList = [9,16,19,20,21,25,26,27,30,43,48,51,52,58,62,64,68,73,82,89,95,97,99,105,107,108,116,125,129,133,138,139]
for i in cellList:
    for j in range(locDF.shape[0]):
        ax.plot(locDF.loc[j,i],alpha=0.5)
    ax.plot(locDF.loc[:,i].mean(),c='k',label = i)
    ax.set_ylim(-0.2, 0.8)
    ax.legend(loc = 'upper right', fontsize = 10)
    plt.pause(1.0)
    ax.cla()

## Correlation Between $\Delta$F/F$_0$ , Locomotion 

In [648]:
relevTime = np.arange(-10,10,1/float(30))
relevInds = np.arange(dF.shape[1]-10*30,dF.shape[1]+10*30)

allCorr = []
shiftInds = []
allPearCor = []
for cells in range(dF.shape[0]):
    allCorr.append([signal.fftconvolve(dF[cells],rSpd,'full')])
    shiftInds.append(np.where(abs(allCorr[cells][0][relevInds]) == max(abs(allCorr[cells][0][relevInds])))[0][0])
    if shiftInds[cells] <= 300:
        tmpInd = abs(shiftInds[cells] - 300)
        tmpP, _ = pearsonr(dF[cells][:-tmpInd],rSpd[tmpInd:])
        allPearCor.append(tmpP)
    else:
        tmpInd = shiftInds[cells] - 300
        tmpP, _ = pearsonr(dF[cells][tmpInd:],rSpd[:-tmpInd])
        allPearCor.append(tmpP)

# fig,ax = plt.subplots(ncols=2)
# for i in range(len(allCorr))[:]:
#     ax[0].plot(relevTime,allCorr[i][0][relevInds],label = i)
# #     ax[0].plot([relevTime[np.where(relevInds == shiftInds[i])], relevTime[np.where(relevInds == shiftInds[i])]],
# #             [0, allCorr[i][0][shiftInds[i]]],c='r')
# #     ax[0].legend(loc = 'upper right', fontsize = 10)
# #     if (dF.shape[1] - shiftInds[i]) < 0:

# #         tmpInd = abs(dF.shape[1] - shiftInds[i])
# #         tmp, _ = pearsonr(dF[i][:-tmpInd],rSpd[tmpInd:])
# #         allPearCorr.append(tmp)
# #         ax[1].scatter(dF[i][:-tmpInd],rSpd[tmpInd:],alpha=0.5)
# #     else:
# #         tmpInd = abs(dF.shape[1] - shiftInds[i])
# #         tmp, _ = pearsonr(dF[i][tmpInd:],rSpd[:-tmpInd])
# #         ax[1].scatter(dF[i][tmpInd:],rSpd[:-tmpInd],alpha=0.5)
#     ax[0].plot([relevTime[shiftInds[i]], relevTime[shiftInds[i]]],[0, allCorr[i][0][relevInds][shiftInds[i]]])
#     ax[1].plot(relevTime, allCorr_shifted[i][0][relevInds])
#     ax[0].set_ylim(0, allCorr[i][0][relevInds][shiftInds[i]]+1000)
#     ax[1].set_ylim(0, allCorr[i][0][relevInds][shiftInds[i]]+1000)
#     plt.pause(1.5)
#     for currAx in ax:
#         currAx.cla()

#### Test cross correlation

In [832]:
x = np.linspace(-np.pi, np.pi, 201)
sin1 = np.sin(x)
cos1 = np.cos(x)
cos2 = np.cos(x-np.pi)
fig,ax = plt.subplots(nrows=2, ncols=2)
ax[0,0].plot(sin1,label='sin')
ax[0,0].plot(cos1,label='cos1')
ax[0,1].plot(sin1,label='sin')
ax[0,1].plot(cos2,label='cos2')
ax[0,0].legend()
ax[0,1].legend()


convSig1 = signal.fftconvolve(cos1,sin1,'full')
shiftAmt = len(x) - np.where(convSig1 == max(convSig1))[0][0]
shiftAmtAbs = abs(shiftAmt)

ax[1,0].plot(convSig1,label='cos1, sin1')
# ax[1,0].plot(signal.fftconvolve(cos1[:-shiftAmtAbs],sin1[shiftAmtAbs:],'full'),label='cos1, sin1[shiftAmtAbs:]')
ax[1,0].legend()
print('shiftAmt', shiftAmt, 'pearsonr uncorrected', pearsonr(cos1,sin1), 
'pearsonr corrected', pearsonr(cos1[:-shiftAmtAbs],sin1[shiftAmtAbs:]))

convSig2 = signal.fftconvolve(cos2,sin1,'full')
shiftAmt = len(x) - np.where(convSig2 == max(convSig2))[0][0]
shiftAmtAbs = abs(shiftAmt)
print('shiftAmt', shiftAmt, 'pearsonr uncorrected', pearsonr(cos2,sin1),
'pearsonr corrected', pearsonr(cos2[shiftAmtAbs:],sin1[:-shiftAmtAbs]))

# convSig2 = signal.fftconvolve(cos2, sin1,'full')
# ax[1].plot(convSig2,label='cos2, sin1')

('shiftAmt', -43, 'pearsonr uncorrected', (6.5205576441784033e-18, 1.0), 'pearsonr corrected', (0.97764934302632223, 1.4409837692439854e-107))
('shiftAmt', 45, 'pearsonr uncorrected', (1.144485170623795e-16, 1.0), 'pearsonr corrected', (0.98868945248137063, 8.3223671071158299e-129))


#### Back to business

In [706]:
pearCor_DF = pd.DataFrame(columns = ['expID','cell_pear_r'], index = np.arange(237))

currExpInd = 0
for expContainerID in expMetaData.id:
    exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
    rSpd, t_rSpd = exp.get_running_speed()
    t_dF, dF = exp.get_dff_traces()

    nanMask = ~np.isnan(rSpd)
    rSpd = rSpd[nanMask]
    rSpd = abs(rSpd)
    dF = dF[:,nanMask]
    
    relevTime = np.arange(-10,10,1/float(30))
    relevInds = np.arange(dF.shape[1]-10*30,dF.shape[1]+10*30)

    allCorr = []
    shiftInds = []
    allPearCor = []
    for cells in range(dF.shape[0]):
        allCorr.append([signal.fftconvolve(dF[cells],rSpd,'full')])
        shiftInds.append(np.where(abs(allCorr[cells][0][relevInds]) == max(abs(allCorr[cells][0][relevInds])))[0][0])
        if shiftInds[cells] == 300:
            tmpP, _ = pearsonr(dF[cells],rSpd)
            allPearCor.append(tmpP)
        elif shiftInds[cells] <= 300:
            tmpInd = abs(shiftInds[cells] - 300)
            tmpP, _ = pearsonr(dF[cells][:-tmpInd],rSpd[tmpInd:])
            allPearCor.append(tmpP)
        else:
            tmpInd = shiftInds[cells] - 300
            tmpP, _ = pearsonr(dF[cells][tmpInd:],rSpd[:-tmpInd])
            allPearCor.append(tmpP)
    pearCor_DF.expID[currExpInd] = expContainerID
    pearCor_DF.cell_pear_r[currExpInd] = allPearCor
    currExpInd += 1

/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 114118 but corresponding boolean dimension is 114119
/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 115732 but corresponding boolean dimension is 115733
/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 105712 but corresponding boolean dimension is 105713
/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 115480 but corresponding boolean dimension is 115481
/Users/bilalbari/anaconda/lib/python2.7/site-packages/ipykernel/

In [863]:
tmp = np.hstack(pearCor_DF.cell_pear_r)
i = 0
while np.size(np.where(np.greater(pearCor_DF.cell_pear_r[i],0.4))) == 0:
    i += 1
cell = np.where(np.greater(pearCor_DF.cell_pear_r[4],0.2))[0][0]

expContainerID = pearCor_DF.iloc[i].expID

exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
rSpd, t_rSpd = exp.get_running_speed()
t_dF, dF = exp.get_dff_traces()

nanMask = ~np.isnan(rSpd)
rSpd = rSpd[nanMask]
rSpd = abs(rSpd)
dF = dF[:,nanMask]

In [852]:
relevTime = np.arange(-10,10,1/float(30))
relevInds = np.arange(dF.shape[1]-10*30,dF.shape[1]+10*30)

fig,ax = plt.subplots()
# ax[0].plot(dF[cell])
# ax[1].plot(rSpd)
convSig = signal.fftconvolve(dF[cell],rSpd,'full')[relevInds]
shiftInd = abs(300 - np.where(convSig == max(convSig))[0])[0]
convSig = signal.fftconvolve(dF[cell][:-shiftInd],rSpd[shiftInd:],'full')
# ax.plot(relevTime,convSig[relevInds])
ax.scatter(dF[cell][:-shiftInd],rSpd[shiftInd:])

In [937]:
pearCor_DF.cell

,expID,cell_pear_r
0,502066273,"[-0.00427375419883, -0.0264708571354, 0.017608..."
1,506278598,"[0.0118918062803, -0.0239296260073, 0.00376906..."
2,510529677,"[-0.02365189443, -0.00383100547428, 0.00875717..."
3,504614224,"[0.0183819620879, 0.0328345179151, 0.021849464..."
4,501724959,"[0.0176216217969, 0.166220846063, 0.0419605520..."
5,506156402,"[0.0573909176377, -0.0152893387942, 0.01429355..."
6,508366303,"[-0.0095634464986, 0.00948527175476, -0.051613..."
7,510093797,"[-0.0380764025415, 0.0519283426752, 0.08500858..."
8,507990552,"[0.101310020352, -0.0434426378451, 0.085483126..."
9,508596945,"[-0.0224853985258, -0.0681081788539, 0.0491872..."


In [ ]:
expContainerID = expMetaData.id[5]
# expContainerID = 501940850
exp = boc.get_ophys_experiment_data(ophys_experiment_id = expContainerID)
rSpd, _ = exp.get_running_speed()
_, dF = exp.get_dff_traces()

# nanMask = ~np.isnan(rSpd)
# rSpd = rSpd[nanMask]
# rSpd = abs(rSpd)
# dF = dF[:,nanMask]
dF = dF[:,1:]
rSpd = mocVec

rSpd_Matx = []
for i in np.arange(-60,60):
    if i < 0:
        rSpd_Matx.append(np.append(np.empty(abs(i))*np.nan,rSpd[:i]))
    elif i == 0:
        rSpd_Matx.append(rSpd)
    elif i > 0:
        rSpd_Matx.append(np.append(rSpd[i:],np.empty(i)*np.nan))

for i in range(dF.shape[0]):
    data = pd.DataFrame({  'x0': rSpd_Matx[  0],   'x1': rSpd_Matx[  1],   'x2': rSpd_Matx[  2],   'x3': rSpd_Matx[  3],   'x4': rSpd_Matx[  4], 
                           'x5': rSpd_Matx[  5],   'x6': rSpd_Matx[  6],   'x7': rSpd_Matx[  7],   'x8': rSpd_Matx[  8],   'x9': rSpd_Matx[  9], 
                          'x10': rSpd_Matx[ 10],  'x11': rSpd_Matx[ 11],  'x12': rSpd_Matx[ 12],  'x13': rSpd_Matx[ 13],  'x14': rSpd_Matx[ 14], 
                          'x15': rSpd_Matx[ 15],  'x16': rSpd_Matx[ 16],  'x17': rSpd_Matx[ 17],  'x18': rSpd_Matx[ 18],  'x19': rSpd_Matx[ 19], 
                          'x20': rSpd_Matx[ 20],  'x21': rSpd_Matx[ 21],  'x22': rSpd_Matx[ 22],  'x23': rSpd_Matx[ 23],  'x24': rSpd_Matx[ 24], 
                          'x25': rSpd_Matx[ 25],  'x26': rSpd_Matx[ 26],  'x27': rSpd_Matx[ 27],  'x28': rSpd_Matx[ 28],  'x29': rSpd_Matx[ 29], 
                          'x30': rSpd_Matx[ 30],  'x31': rSpd_Matx[ 31],  'x32': rSpd_Matx[ 32],  'x33': rSpd_Matx[ 33],  'x34': rSpd_Matx[ 34], 
                          'x35': rSpd_Matx[ 35],  'x36': rSpd_Matx[ 36],  'x37': rSpd_Matx[ 37],  'x38': rSpd_Matx[ 38],  'x39': rSpd_Matx[ 39], 
                          'x40': rSpd_Matx[ 40],  'x41': rSpd_Matx[ 41],  'x42': rSpd_Matx[ 42],  'x43': rSpd_Matx[ 43],  'x44': rSpd_Matx[ 44], 
                          'x45': rSpd_Matx[ 45],  'x46': rSpd_Matx[ 46],  'x47': rSpd_Matx[ 47],  'x48': rSpd_Matx[ 48],  'x49': rSpd_Matx[ 49], 
                          'x50': rSpd_Matx[ 50],  'x51': rSpd_Matx[ 51],  'x52': rSpd_Matx[ 52],  'x53': rSpd_Matx[ 53],  'x54': rSpd_Matx[ 54], 
                          'x55': rSpd_Matx[ 55],  'x56': rSpd_Matx[ 56],  'x57': rSpd_Matx[ 57],  'x58': rSpd_Matx[ 58],  'x59': rSpd_Matx[ 59],
                          'x60': rSpd_Matx[ 60],  'x61': rSpd_Matx[ 61],  'x62': rSpd_Matx[ 62],  'x63': rSpd_Matx[ 63],  'x64': rSpd_Matx[ 64],
                          'x65': rSpd_Matx[ 65],  'x66': rSpd_Matx[ 66],  'x67': rSpd_Matx[ 67],  'x68': rSpd_Matx[ 68],  'x69': rSpd_Matx[ 69],
                          'x70': rSpd_Matx[ 70],  'x71': rSpd_Matx[ 71],  'x72': rSpd_Matx[ 72],  'x73': rSpd_Matx[ 73],  'x74': rSpd_Matx[ 74],
                          'x75': rSpd_Matx[ 75],  'x76': rSpd_Matx[ 76],  'x77': rSpd_Matx[ 77],  'x78': rSpd_Matx[ 78],  'x79': rSpd_Matx[ 79],
                          'x80': rSpd_Matx[ 80],  'x81': rSpd_Matx[ 81],  'x82': rSpd_Matx[ 82],  'x83': rSpd_Matx[ 83],  'x84': rSpd_Matx[ 84],
                          'x85': rSpd_Matx[ 85],  'x86': rSpd_Matx[ 86],  'x87': rSpd_Matx[ 87],  'x88': rSpd_Matx[ 88],  'x89': rSpd_Matx[ 89],
                          'x90': rSpd_Matx[ 90],  'x91': rSpd_Matx[ 91],  'x92': rSpd_Matx[ 92],  'x93': rSpd_Matx[ 93],  'x94': rSpd_Matx[ 94],
                          'x95': rSpd_Matx[ 95],  'x96': rSpd_Matx[ 96],  'x97': rSpd_Matx[ 97],  'x98': rSpd_Matx[ 98],  'x99': rSpd_Matx[ 99],
                         'x100': rSpd_Matx[100], 'x101': rSpd_Matx[101], 'x102': rSpd_Matx[102], 'x103': rSpd_Matx[103], 'x104': rSpd_Matx[104],
                         'x105': rSpd_Matx[105], 'x106': rSpd_Matx[106], 'x107': rSpd_Matx[107], 'x108': rSpd_Matx[103], 'x109': rSpd_Matx[109],
                         'x110': rSpd_Matx[110], 'x111': rSpd_Matx[111], 'x112': rSpd_Matx[112], 'x113': rSpd_Matx[113], 'x114': rSpd_Matx[114],
                         'x115': rSpd_Matx[115], 'x116': rSpd_Matx[116], 'x117': rSpd_Matx[117], 'x118': rSpd_Matx[118], 'x119': rSpd_Matx[119],
                           'y': dF[i]})                   

    # data = pd.DataFrame({'x1': rSpd, 'x2': rSpd_Matx[1], 'y': dF[0]})
    model = model = ols("y ~ x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12 + x13 + x14 + x15 + "
                        "x16 + x17 + x18 + x19 + x20 + x21 + x22 + x23 + x24 + x25 + x26 + x28 + x29 + x30 + x31 + "
                        "x32 + x33 + x34 + x35 + x36 + x37 + x38 + x39 + x40 + x41 + x42 + x43 + x44 + x45 + x46 + "
                        "x47 + x48 + x49 + x50 + x51 + x52 + x53 + x54 + x55 + x56 + x57 + x58 + x59 + x60 + x61 + "
                        "x62 + x63 + x64 + x65 + x66 + x67 + x68 + x69 + x70 + x71 + x72 + x73 + x74 + x75 + x76 + "
                        "x77 + x78 + x79 + x80 + x81 + x82 + x83 + x84 + x85 + x86 + x87 + x88 + x89 + x90 + x91 + "
                        "x92 + x93 + x94 + x95 + x96 + x97 + x98 + x99 + x100 + x101 + x102 + x103 + x104 + x105 + "
                        "x106 + x107 + x108 + x109 + x110 + x111 + x112 + x113 + x114 + x115 + x116 + x117 + x118 + "
                        "x119", data).fit()
    print("%.5f, %.5f" %(model.rsquared, model.rsquared_adj))